# Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

rng = np.random.RandomState(42)

# Data

In [ ]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

print(data.shape)
data.head()

(40778, 54)


,DATE,TARGET,TMAX_d1,TMAX_d1_d2_diff,TMAX_3day_mean,TMAX_3day_std,TMAX_3day_min,TMAX_3day_max,TMAX_3day_median,TMAX_5day_mean,...,MONTH_11,MONTH_12,STATION_USW00003967,STATION_USW00012916,STATION_USW00013960,STATION_USW00024025,STATION_USW00024233,STATION_USW00093067,STATION_USW00093225,STATION_USW00094728
0,2010-01-16,15.6,15.0,-1.1,13.700000,3.251154,10.0,16.1,15.0,12.98,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-17,14.4,15.6,0.6,15.566667,0.550757,15.0,16.1,15.6,14.22,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-01-18,17.2,14.4,-1.2,15.000000,0.600000,14.4,15.6,15.0,14.22,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-01-19,21.1,17.2,2.8,15.733333,1.404754,14.4,17.2,15.6,15.66,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-20,23.9,21.1,3.9,17.566667,3.365016,14.4,21.1,17.2,16.66,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [ ]:
model = DecisionTreeRegressor(random_state=rng)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 4.208539051501395e-17
Valid MAE: 4.603424657534247


# Hyperparameter tuning

In [ ]:
param_grid = {
    'max_depth': [None, 12, 10, 8, 6, 5, 4, 3, 2],
    'min_samples_leaf': [1, 10, 25, 50, 100, 200, 500, 1000],
    'min_impurity_decrease': [0, 0.01, 0.1],
    'ccp_alpha': [0, 0.0005, 0.001, 0.005, 0.01, 0.1]
}
model = DecisionTreeRegressor(random_state=rng)
grid = GridSearchCV(
    model, param_grid, cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)
grid.fit(X_train, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
Best score:  -3.0842241640071264
Best params:  {'ccp_alpha': 0.005, 'max_depth': 10, 'min_impurity_decrease': 0, 'min_samples_leaf': 100}


In [ ]:
model = DecisionTreeRegressor(
    max_depth=10, min_samples_leaf=100,
    ccp_alpha=0.005,
    random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.9478319678886105
Valid MAE: 3.236205953878172


In [ ]:
model = DecisionTreeRegressor(
    max_depth=10, min_samples_leaf=100,
    # ccp_alpha=0.005,
    random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.91724912637101
Valid MAE: 3.256837206853774
